In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Upload data
df = pd.read_csv('../data/data_2011_2024.csv')
#df.info()

In [ ]:
# Create a datetime column for plotting
df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))

In [ ]:
# Plot time series of climate variables and dengue cases
f, ax = plt.subplots(5, 1, figsize=(10, 10), sharex=True)
ax = ax.flatten()

cvariables = ['tavg', 'tmax', 'tmin', 'wind_speed', 'precip']
titles = ['Average Temperature', 'Maximum Temperature', 'Minimum Temperature', 'Wind Speed', 'Precipitation']
units = ['°C', '°C', '°C', 'm/s', 'mm']

for i, variable in enumerate(cvariables):
    # For each county, get the mean value of the variable for each date
    county_means = df.groupby(['date', 'county'])[variable].mean().reset_index()
    
    # Create a pivot table with dates as index and counties as columns
    pivot_data = county_means.pivot(index='date', columns='county', values=variable)
    
    # Plot each county as a separate line
    pivot_data.plot(ax=ax[i], legend=False, alpha=0.3, linewidth=0.5)
    
    # Plot the overall mean as a thicker line
    overall_mean = df.groupby('date')[variable].mean()
    overall_mean.plot(ax=ax[i], color='black', linewidth=1)
    
    ax[i].set_title(titles[i]+' ('+units[i]+')', fontsize=10)

# Add a common legend for the first plot
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles[:1] + [handles[-1]], ['Individual counties', 'Overall mean'], loc='upper right', ncol=2)

# Set x-axis labels to show all years
unique_years = sorted(df['year'].unique())
unique_years = [year for year in unique_years if year != 2011]  # Exclude 2011
ax[-1].set_xticks(pd.to_datetime([f'{year}-01-01' for year in unique_years]))
ax[-1].set_xticklabels(unique_years)

plt.tight_layout()

In [ ]:
# Create df with climate variables and case counts aggregated by month
ts = pd.DataFrame()
for var in ['case_count', 'tavg', 'tmax', 'tmin', 'wind_speed', 'precip']:
    # Summarise cases
    if var == 'case_count':
        ts[var] = df.groupby('date')[var].sum()
    # Mean of climatic variables
    else:
        ts[var] = df.groupby('date')[var].mean()

f, ax1 = plt.subplots(figsize=(15, 5))
line1 = ts['tavg'].plot(color='purple', ax=ax1, label='Average Temperature')
line2 = ts['tmax'].plot(color='red', ax=ax1, label='Maximum Temperature')
line3 = ts['tmin'].plot(color='lightblue', ax=ax1, label='Minimum Temperature')

ax2 = ax1.twinx()
line4 = ts['precip'].plot(color='darkblue', ax=ax2, label='Precipitation')

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Set labels for both y-axes
ax1.set_ylabel('Temperature')
ax2.set_ylabel('Precipitation')
ax1.set_xlabel('Date')

plt.tight_layout()

In [ ]:
ts.corr(method='spearman')

In [ ]:
# Create correlation matrix
plt.figure(figsize=(8, 6))
corr_matrix = ts.corr(method='spearman')
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Between Variables (Monthly Averages)')
plt.tight_layout()

In [ ]:
# Calculate lag for each climate variable
for i in range(5):
    for lag in range(6):
        ts[f'{cvariables[i]}_lag{lag+1}'] = ts[cvariables[i]].shift(periods=lag+1)

In [ ]:
variables_tavg = ['case_count', 'tavg', 'tavg_lag1', 'tavg_lag2', 'tavg_lag3', 'tavg_lag4', 'tavg_lag5', 'tavg_lag6']
# Create correlation matrix
plt.figure(figsize=(10, 8))
corr_matrix = ts[variables_tavg].corr(method='spearman')
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Between Variables (Monthly Averages)')
plt.tight_layout()

In [ ]:
# Exclude 2011 after calculating lag
ts = ts[ts.index.year != 2011]

# Prepare formulas
c_formula = "case_count ~ tavg + tmax + tmin + wind_speed + precip"
c_formula_lag1 = "case_count ~ tavg_lag1 + tmax_lag1 + tmin_lag1 + wind_speed_lag1 + precip_lag1"
c_formula_lag2 = "case_count ~ tavg_lag2 + tmax_lag2 + tmin_lag2 + wind_speed_lag2 + precip_lag2"

# Model 1: Basic model
nbi_model = smf.glm(
    formula=c_formula, 
    data=ts,
    family=sm.families.NegativeBinomial()
)
results1 = nbi_model.fit()
print("\nNegative Binomial Model Results (without lag):")
print(results1.summary())

# Model 2: Basic model + lag1
nbi_model = smf.glm(
    formula=c_formula_lag1, 
    data=ts,
    family=sm.families.NegativeBinomial()
)
results2 = nbi_model.fit()
print("\nNegative Binomial Model Results (with 1-month lag):")
print(results2.summary())

# Model 3: Basic model + lag2
nbi_model = smf.glm(
    formula=c_formula_lag2, 
    data=ts,
    family=sm.families.NegativeBinomial()
)
results3 = nbi_model.fit()
print("\nNegative Binomial Model Results (with 2-month lag):")
print(results3.summary())

In [ ]:
# Examine residuals for model diagnostics
results = [results1, results2, results3]
model_names = ['Negative Binomial Model (without lag)', 'Negative Binomial Model (with 1-month lag)', 'Negative Binomial Model (with 2-month lag)']

for i, result in enumerate(results):
    f, ax = plt.subplots(1, 4, figsize=(20, 5))

    # Generate predicted values
    ts['predicted'] = result.predict()

    # Residuals
    ax[0].scatter(ts['predicted'], ts['case_count'] - ts['predicted'], alpha=.5)
    ax[0].axhline(y=0, color='r', linestyle='-')
    ax[0].set_title('Residuals vs Predicted')
    ax[0].set_xlabel('Predicted values')
    ax[0].set_ylabel('Residuals')

    # Actual vs predicted
    ax[1].scatter(ts['case_count'], ts['predicted'], alpha=.5)
    max_val = max(ts['case_count'].max(), ts['predicted'].max())
    ax[1].plot([0, max_val], [0, max_val], 'r--')
    ax[1].set_title('Predicted vs Actual values')
    ax[1].set_xlabel('Actual values')
    ax[1].set_ylabel('Predicted values')

    # Q-Q plot
    sm.qqplot(ts['case_count'] - ts['predicted'], line='45', ax=ax[2], alpha=.5)
    ax[2].set_title('Q-Q plot of residuals')

    # Histogram of residuals
    ax[3].hist(ts['case_count'] - ts['predicted'], bins=20)
    ax[3].set_title('Histogram of residuals')
    ax[3].set_xlabel('Residuals')

    plt.suptitle(f"Diagnostics for {model_names[i]}", fontsize=16)
    plt.tight_layout()